In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
#from torchvision import datasets, transforms

from typing import Tuple, List
from collections import deque

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Optimizer

import numpy as np
from torch import Tensor

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
torch.set_default_device(device)

Using cpu device


In [4]:
torch.manual_seed(20190325);

In [100]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :1]])


In [118]:
print(raw_df.values[1::2, 1:3])

[[ 4.98 24.  ]
 [ 9.14 21.6 ]
 [ 4.03 34.7 ]
 ...
 [ 5.64 23.9 ]
 [ 6.48 22.  ]
 [ 7.88 11.9 ]]


In [102]:
print(data[0])

[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02]


In [119]:
target = raw_df.values[1::2, 1:3]
print(target)

[[ 4.98 24.  ]
 [ 9.14 21.6 ]
 [ 4.03 34.7 ]
 ...
 [ 5.64 23.9 ]
 [ 6.48 22.  ]
 [ 7.88 11.9 ]]


In [125]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

#y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)
y_train, y_test = Tensor(y_train), Tensor(y_test)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

In [ ]:
print(y_train)

In [204]:
class HousePricesModel(nn.Module):
  def __init__(self, hidden_size: int = 12):
    super().__init__()
    self.fc1 = nn.Linear(12, 24)
    self.fc2 = nn.Linear(24, 24)
    self.fc3 = nn.Linear(24, 12)
    self.fc4 = nn.Linear(12, 2)

  def forward(self, x: Tensor) -> Tensor:
    #assert_dim(x, 2)
    assert x.shape[1] == 12
    x = self.fc1(x)
    x = torch.sigmoid(x)
    x = self.fc2(x)
    x = torch.sigmoid(x)
    x = self.fc3(x)
    x = torch.sigmoid(x)

    return self.fc4(x)

In [205]:
model = HousePricesModel().to(device)
print(model)

HousePricesModel(
  (fc1): Linear(in_features=12, out_features=24, bias=True)
  (fc2): Linear(in_features=24, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=12, bias=True)
  (fc4): Linear(in_features=12, out_features=2, bias=True)
)


Testing if NN gives two numbers in prediction

In [182]:
print(X_test[0])
print(X_test[0].unsqueeze(0))
print(X_test[0].unsqueeze(0).shape)
net = HousePricesModel()
res = net(X_test[0].unsqueeze(0))
print(res)

tensor([-0.4164,  3.5896, -1.2339, -0.2726, -1.1972,  2.2344, -1.2580,  0.6289,
        -0.6380, -1.0942, -1.7364,  0.3959])
tensor([[-0.4164,  3.5896, -1.2339, -0.2726, -1.1972,  2.2344, -1.2580,  0.6289,
         -0.6380, -1.0942, -1.7364,  0.3959]])
torch.Size([1, 12])
tensor([[-0.3640, -0.3124]], grad_fn=<AddmmBackward0>)


In [183]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [184]:
criterion = nn.MSELoss()

In [185]:
print(X_train[0])
print(y_train[0])

tensor([-0.4125, -0.4877, -1.3069, -0.2726, -0.8353,  1.2286, -0.5112,  1.0777,
        -0.7529, -1.1061,  0.1130,  0.4411])
tensor([ 5.3300, 36.2000])


Trainer code. Might be will need modifications for different output

In [186]:
class PyTorchTrainer(object):

  def __init__(self, model: nn.Module, optim: Optimizer, criterion: _Loss):
    self.model = model
    self.optim = optim
    self.loss = criterion
    self._check_optim_net_aligned()

  def _check_optim_net_aligned(self):
    assert self.optim.param_groups[0]['params'] == list(self.model.parameters())

  def _generate_batches(self, X: Tensor, y: Tensor, size: int = 32) -> Tuple[Tensor]:
    N = X.shape[0]
    for ii in range(0, N, size):
      X_batch, y_batch = X[ii:ii+size], y[ii:ii+size]
      yield X_batch, y_batch

  def fit(self, X_train: Tensor, y_train: Tensor, X_test: Tensor, y_test: Tensor,epochs: int=100,eval_every: int=10, batch_size: int=32):
    for e in range(epochs):
      #X_train, y_train = permute_data(X_train, y_train)
      batch_generator = self._generate_batches(X_train, y_train, batch_size)

      for ii, (X_batch, y_batch) in enumerate(batch_generator):
        #print(X_batch.device)
        self.optim.zero_grad()
        output = self.model(X_batch)
        loss = self.loss(output, y_batch)
        loss.backward()
        self.optim.step()
      output = self.model(X_test)
      loss = self.loss(output, y_test)
      print(e, loss)

In [ ]:
net = HousePricesModel()
optimizer = optim.SGD(net.parameters(), lr=0.0002)#, momentum=0.7)
criterion = nn.MSELoss()

trainer = PyTorchTrainer(net, optimizer, criterion)
#print(y_test.device)
trainer.fit(X_train, y_train, X_test, y_test, epochs=30000, eval_every=200)

In [199]:
print(X_train[2])
print(y_train[2])

tensor([ 9.9339, -0.4877,  1.0160, -0.2726,  1.0047,  0.9736,  0.8295, -1.1307,
         1.6612,  1.5309,  0.8066,  0.4411])
tensor([17.2100, 10.4000])


In [200]:
res = net(X_test[2].unsqueeze(0))
print(res)

tensor([[15.1058, 17.4038]], grad_fn=<AddmmBackward0>)
